In [1]:
import pyspark 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[2]') \
    .appName('test') \
    .getOrCreate()

In [2]:
!tree

.
|-- 06_spark_sql.ipynb
|-- fhvhv_tripdata_2021-06.csv.gz
`-- readme.md

0 directories, 3 files


In [15]:
df = spark.read.option("header",True).option('inferSchema',True).csv('fhvhv_tripdata_2021-06.csv.gz')


In [16]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [19]:
print(f'Number of columns: {len(df.columns)}')

Number of columns: 7


In [20]:
print(f'Number of rows: {df.count()}')

Number of rows: 14961892


# Question 2

In [21]:
df.repartition(12).write.parquet('Q2',mode='overwrite')

# Question 3

> Count records\
> How many taxi trips were there on June 15?\
> Consider only trips that started on June 15.

In [24]:
from pyspark.sql import functions as F

In [34]:
df = df.withColumn('pickup_day', F.dayofmonth('pickup_datetime'))

In [32]:
df.filter(df.pickup_day  == 15).count()

452470

In [37]:
df.head(1)

[Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 2, 41), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 7, 46), PULocationID=174, DOLocationID=18, SR_Flag='N', Affiliated_base_number='B02764', pickup_day=1)]

In [56]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)
 |-- pickup_day: integer (nullable = true)
 |-- trip_duration: integer (nullable = true)



In [71]:
spark.sql('SELECT  max(cast(dropoALXD0001 - S&T (Feb 22 - )/Project Alexandria - Snacks & Treats Main Log (Update) [Replacing 9548] - Level 1ff_datetime as int) - cast(pickup_datetime as int))/(60*60) trip_duration FROM data ').show()

+----------------+
|   trip_duration|
+----------------+
|66.8788888888889|
+----------------+



In [90]:
from pyspark.sql.types import IntegerType
df.withColumn('trip_duration', df.dropoff_datetime.cast(IntegerType()) - df.pickup_datetime.cast(IntegerType())).select(F.max('trip_duration')/(60*60)).show()

+---------------------------+
|(max(trip_duration) / 3600)|
+---------------------------+
|           66.8788888888889|
+---------------------------+



# Question 4

In [92]:
df_zone = spark.read.option('header',True).option('inferSchema',True).csv('taxi_zone_lookup.csv')

In [93]:
df_zone.show(1)

+----------+-------+--------------+------------+
|LocationID|Borough|          Zone|service_zone|
+----------+-------+--------------+------------+
|         1|    EWR|Newark Airport|         EWR|
+----------+-------+--------------+------------+
only showing top 1 row



In [95]:
cb_df = df.join(df_zone, df.PULocationID == df_zone.LocationID, how='inner')

In [96]:
cb_df.head(1)

[Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 2, 41), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 7, 46), PULocationID=174, DOLocationID=18, SR_Flag='N', Affiliated_base_number='B02764', pickup_day=1, trip_duration=0, LocationID=174, Borough='Bronx', Zone='Norwood', service_zone='Boro Zone')]

In [106]:
from pyspark.sql.functions import col

cb_df.groupBy('Zone').count().sort(col('count').desc()).show()

+--------------------+------+
|                Zone| count|
+--------------------+------+
| Crown Heights North|231279|
|        East Village|221244|
|         JFK Airport|188867|
|      Bushwick South|187929|
|       East New York|186780|
|TriBeCa/Civic Center|164344|
|   LaGuardia Airport|161596|
|            Union Sq|158937|
|        West Village|154698|
|             Astoria|152493|
|     Lower East Side|151020|
|        East Chelsea|147673|
|Central Harlem North|146402|
|Williamsburg (Nor...|143683|
|          Park Slope|143594|
|  Stuyvesant Heights|141427|
|        Clinton East|139611|
|West Chelsea/Huds...|139431|
|             Bedford|138428|
|         Murray Hill|137879|
+--------------------+------+
only showing top 20 rows

